In [ ]:
# !pip install tensorflow
# !pip install pandas
# !pip install sweetviz
# !pip install ydata-profiling
# !pip install ipywidgets

In [2]:
# import sweetviz
# sweetviz.__version__

In [3]:
import os
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# import tensorflow as tf
import json
import torch
import segmentation_models_pytorch as smp
import os
import numpy as np
import json
# from keras.preprocessing import image
import matplotlib.pyplot as plt
from tqdm import tqdm
# import seaborn as sns
import sweetviz as sv
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
pd.__version__

'1.5.3'

In [5]:
pd.options.display.max_rows = 100
pd.options.display.max_columns = 100

In [6]:
def read_json(json_path):
    with open(json_path, 'r') as f:
        data = json.load(f)
    f.close()
    # Create different lists from the data dictionary

    annotations = pd.DataFrame(data["annotations"])
    category = pd.DataFrame(data["categories"])
    info = pd.DataFrame.from_dict(data["info"], orient='index')
    images = pd.DataFrame(data['images'])
    licenses = pd.DataFrame(data['licenses'])
    return annotations, category, info, images, licenses

In [7]:
json_train_path = './dataset/fungi/train_val_annotations/train.json'
json_val_path = './dataset/fungi/train_val_annotations/val.json'
tr_annotation, tr_category, tr_info, tr_images, tr_licenses = read_json(json_train_path)
val_annotation, val_category, val_info, val_images, val_licenses = read_json(json_val_path)

In [8]:
tr_annotation.head()

,id,category_id,image_id
0,43829,650,21942
1,44039,650,21943
2,44255,650,21944
3,44614,650,22210
4,45510,650,24812


In [9]:
tr_annotation['category_id'].nunique()

1394

In [18]:
categories = sorted([int(i) for i in tr_annotation['category_id'].unique().tolist()])
categories[:3]

[0, 1, 2]

In [11]:
def create_merged_json_df(images, annotations, category):
    """ args = takes the three important dataframes
    returns = create a merged dataframe with all the important data
    """
    data_df = images.copy()

    ## to the images dataframe, we add the category_id column - our target
    data_df['category_id'] = annotations[annotations['image_id'] == data_df['id']]['category_id']
    category.set_index('id')
    data_df = pd.merge(left=data_df, right=category, how='left', left_on='category_id', right_on='id')
    data_df.drop(columns = ['id_y'], inplace=True)
    #change the column to a string?
    data_df['category_id'] = data_df['category_id'].astype(str) 
    # Add the columns of image name and its subdirectory/catefory
    data_df['image_name'] = data_df['file_name'].str.split('/').str[2]
    data_df['subdir'] = data_df['file_name'].str.split(pat='/', n=1).str[1]
    return data_df

In [13]:
tr_data = create_merged_json_df(tr_images, tr_annotation, tr_category)
tr_data['freq'] = tr_data.groupby('category_id')['category_id'].transform('count')
# tr_data['category_id'] = tr_annotation[tr_annotation['image_id'] == tr_data['id']]['category_id']
# tr_data.sort_values(by=['file_name'], inplace=True)
tr_data.head(3)

,id_x,file_name,width,license,rights_holder,height,category_id,name,supercategory,image_name,subdir,freq
0,21942,images/16390_Leucoagaricus_leucothites/OBL2010...,1300,1,Ole B. Lyshede,732,650,Leucoagaricus leucothites,Leucoagaricus,OBL2010PIC27332400.JPG,16390_Leucoagaricus_leucothites/OBL2010PIC2733...,81
1,21943,images/16390_Leucoagaricus_leucothites/OBL2010...,1400,1,Ole B. Lyshede,788,650,Leucoagaricus leucothites,Leucoagaricus,OBL2010PIC74504431.JPG,16390_Leucoagaricus_leucothites/OBL2010PIC7450...,81
2,21944,images/16390_Leucoagaricus_leucothites/OBL2010...,1400,1,Ole B. Lyshede,788,650,Leucoagaricus leucothites,Leucoagaricus,OBL2010PIC48004850.JPG,16390_Leucoagaricus_leucothites/OBL2010PIC4800...,81


In [43]:
tr_data[tr_data['category_id']=='650'].shape

(81, 12)

In [21]:
category_id_train = sorted([int(i) for i in tr_data['category_id'].unique().tolist()])
category_id_train[:3]

[0, 1, 2]

In [22]:
tr_data.iloc[0,1]

'images/16390_Leucoagaricus_leucothites/OBL2010PIC27332400.JPG'

In [23]:
val_data = create_merged_json_df(val_images, val_annotation, val_category)
val_data.head(3)

,id_x,file_name,width,license,rights_holder,height,category_id,name,supercategory,image_name,subdir
0,12208,images/16390_Leucoagaricus_leucothites/JM2009P...,1500,1,Jens Mårbjerg,985,650,Leucoagaricus leucothites,Leucoagaricus,JM2009PIC63194578.JPG,16390_Leucoagaricus_leucothites/JM2009PIC63194...
1,58713,images/16390_Leucoagaricus_leucothites/JM2012P...,1000,1,Jens Mårbjerg,669,650,Leucoagaricus leucothites,Leucoagaricus,JM2012PIC41096317.JPG,16390_Leucoagaricus_leucothites/JM2012PIC41096...
2,14255,images/16390_Leucoagaricus_leucothites/MC2009P...,700,1,Morten Christensen,469,650,Leucoagaricus leucothites,Leucoagaricus,MC2009PIC47538806.JPG,16390_Leucoagaricus_leucothites/MC2009PIC47538...


In [24]:
tr_category.set_index('id').head()

,name,supercategory
id,,
650,Leucoagaricus leucothites,Leucoagaricus
1158,Byssomerulius corium,Byssomerulius
651,Leucocoprinus brebissonii,Leucocoprinus
652,Leucogyrophana mollusca,Leucogyrophana
1159,Calocybe carnea,Calocybe


In [ ]:
# images_report = sv.analyze(tr_images)
# #display the report
# images_report.show_html('sweetviz_train_images.html')

In [ ]:
# train_data_report = sv.analyze(tr_data)
# #display the report
# train_data_report.show_html('sweetviz_train_data.html')

In [19]:
# from ydata_profiling import ProfileReport
# profile = ProfileReport(tr_images, title='images Pandas Profiling Report')
# # profile

In [20]:
# profile.to_file('Report_train_json.html')

In [25]:
train_images_names = tr_data['file_name'].to_list()
val_images_names = val_data['file_name'].to_list()
len(train_images_names), len(val_images_names)

(85578, 4182)

In [34]:
train_images_cat = tr_data['category_id'].to_list()
val_images_cat = val_data['category_id'].to_list()
len(train_images_cat), len(val_images_cat)

(85578, 4182)

In [36]:
train_key_value_lst = [[key, value] for key, value in zip(train_images_cat, train_images_names)]
val_key_value_lst = [[key, value] for key, value in zip(val_images_cat, val_images_names)]
train_key_value_lst[:3]

[['650', 'images/16390_Leucoagaricus_leucothites/OBL2010PIC27332400.JPG'],
 ['650', 'images/16390_Leucoagaricus_leucothites/OBL2010PIC74504431.JPG'],
 ['650', 'images/16390_Leucoagaricus_leucothites/OBL2010PIC48004850.JPG']]

In [38]:
train_key_value_lst[-3:]

[['649',
  'images/16382_Leucoagaricus_leucothites var. carneifolius/WE2013PIC82590980.JPG'],
 ['649',
  'images/16382_Leucoagaricus_leucothites var. carneifolius/KMO2016-9181394_rkuEVUwge.JPG'],
 ['649',
  'images/16382_Leucoagaricus_leucothites var. carneifolius/KMO2016-9181394_HkeuE4UPxx.JPG']]

In [37]:
val_key_value_lst[:3]

[['650', 'images/16390_Leucoagaricus_leucothites/JM2009PIC63194578.JPG'],
 ['650', 'images/16390_Leucoagaricus_leucothites/JM2012PIC41096317.JPG'],
 ['650', 'images/16390_Leucoagaricus_leucothites/MC2009PIC47538806.JPG']]

In [39]:
val_key_value_lst[-3:]

[['649',
  'images/16382_Leucoagaricus_leucothites var. carneifolius/KNI2009PIC72916487.JPG'],
 ['649',
  'images/16382_Leucoagaricus_leucothites var. carneifolius/KNI2009PIC52108197.JPG'],
 ['649',
  'images/16382_Leucoagaricus_leucothites var. carneifolius/KNI2009PIC22904040.JPG']]

In [41]:
with open('./dataset/fungi/train_images_cat.txt', 'w') as f:
    for line in train_key_value_lst:
        f.write(f"{line[0]}@:@" + "/".join(line[1].split('/')[1:]) + '\n')
f.close()

In [42]:
with open('./dataset/fungi/val_images_cat.txt', 'w') as f:
    for line in val_key_value_lst:
        f.write(f"{line[0]}@:@" + "/".join(line[1].split('/')[1:]) + '\n')
f.close()

In [32]:
with open('./dataset/fungi/val_images.txt', 'w') as f:
    for line in val_images_names:
        f.write("/".join(line.split('/')[1:]) + '\n')
f.close()